In [2]:
import torch
import cv2
import os
import numpy as np
import shutil
from google.colab.patches import cv2_imshow
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import math
from PIL import Image
import torch.nn as nn
import yaml
import random
from google.colab import files
import sys
import time
from torch.utils.data import random_split
import matplotlib.pyplot as plt
device = "cuda" if torch.cuda.is_available() else "cpu" # 檢查是否有可用的 CUDA 設備（通常是顯卡，支援 GPU 運算），如果有，就將 device 變數設置為 "cuda"，否則設置為 "cpu"。
from __future__ import print_function
import zipfile
import os
import pdb
import torch
import h5py
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision import datasets, transforms, utils
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms, utils
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid

from torch.autograd import Variable
# from logger import Logger
import pdb
import os
import re
import numpy as np
import time


import torch
import cv2
import os
# os.chdir('/content/drive/MyDrive/Colab Notebooks/Simple_DE') # this path is the path of the current .ipynb
import numpy as np
import shutil
from google.colab.patches import cv2_imshow
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import math
from PIL import Image
import torch.nn as nn
import yaml
import random
from google.colab import files
import sys
import time
from torch.utils.data import random_split
import matplotlib.pyplot as plt
device = "cuda" if torch.cuda.is_available() else "cpu" # 檢查是否有可用的 CUDA 設備（通常是顯卡，支援 GPU 運算），如果有，就將 device 變數設置為 "cuda"，否則設置為 "cpu"。
from torch.optim.lr_scheduler import LambdaLR

In [3]:
output_height=64
output_width=64
batch_size = 32

In [4]:
class TransposeDepthInput(object):
    def __call__(self, depth):
        depth = depth.transpose((2, 0, 1))
        depth = torch.from_numpy(depth)
        depth = depth.view(1, depth.shape[0], depth.shape[1], depth.shape[2])
        depth = nn.functional.interpolate(depth, size=(output_height, output_width), mode='bilinear', align_corners=False)
        # depth = torch.log(depth)
        return depth[0]

In [5]:
rgb_data_transforms = transforms.Compose([
    transforms.Resize((output_height, output_width)),    # Different for Input Image & Depth Image
    transforms.ToTensor(),
    ])

In [6]:
depth_data_transforms = transforms.Compose([
    TransposeDepthInput(),
])

In [7]:
input_for_plot_transforms = transforms.Compose([
    transforms.Resize((output_height, output_width)),    # Different for Input Image & Depth Image
    transforms.ToTensor(),
])

In [8]:
class NYUDataset(Dataset):
    def calculate_mean(self, images):
        mean_image = np.mean(images, axis=0)
        return mean_image

    def __init__(self, filename, type, rgb_transform = None, depth_transform = None):
        f = h5py.File(filename, 'r')
        if type == "training":
            self.images = f['images'][0:1024]
            self.depths = f['depths'][0:1024]
        elif type == "validation":
            self.images = f['images'][1024:1248]
            self.depths = f['depths'][1024:1248]
        elif type == "test":
            self.images = f['images'][1248:]
            self.depths = f['depths'][1248:]
        self.rgb_transform = rgb_transform
        self.depth_transform = depth_transform
        self.mean_image = self.calculate_mean(f['images'][0:1449])

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        # image = (image - self.mean_image)/np.std(image)
        image = image.transpose((2, 1, 0))
        # image = (image - image.min())/(image.max() - image.min())
        # image = image * 255
        # image = image.astype('uint8')
        image = Image.fromarray(image)
        if self.rgb_transform:
            image = self.rgb_transform(image)
        depth = self.depths[idx]
        depth = np.reshape(depth, (1, depth.shape[0], depth.shape[1]))
        depth = depth.transpose((2, 1, 0))
        if self.depth_transform:
            depth = self.depth_transform(depth)
        sample = {'image': image, 'depth': depth}
        return sample

In [9]:
path = '/content/drive/MyDrive/Colab Notebooks/共用區/Simple_DE/nyu_depth_v2_labeled.mat' # for free account
path = '/content/drive/MyDrive/Colab Notebooks/Simple_DE/nyu_depth_v2_labeled.mat' # for main account
train_loader = torch.utils.data.DataLoader(NYUDataset( path,
                                                       'training',
                                                        rgb_transform = rgb_data_transforms,
                                                        depth_transform = depth_data_transforms),
                                            batch_size = batch_size,
                                            shuffle = True, num_workers = 5)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [10]:
val_loader = torch.utils.data.DataLoader(NYUDataset( path,
                                                       'validation',
                                                        rgb_transform = rgb_data_transforms,
                                                        depth_transform = depth_data_transforms),
                                            batch_size = batch_size,
                                            shuffle = False, num_workers = 5)




In [11]:
class UNetConvBlock(nn.Module):
    def __init__(self, in_size, out_size, kernel_size=3, activation=F.relu):
        super(UNetConvBlock, self).__init__()
        self.conv = nn.Conv2d(in_size, out_size, kernel_size, padding=1)
        self.conv2 = nn.Conv2d(out_size, out_size, kernel_size, padding=1)
        self.activation = activation

    def forward(self, x):
        out = self.activation(self.conv(x))
        out = self.activation(self.conv2(out))
        return out

In [12]:
class UNetUpBlock(nn.Module):
    def __init__(self, in_size, out_size, kernel_size=3, activation=F.relu, space_dropout=False):
        super(UNetUpBlock, self).__init__()
        self.up = nn.ConvTranspose2d(in_size, out_size, 2, stride=2)
        self.conv = nn.Conv2d(in_size, out_size, kernel_size, padding=1)
        self.conv2 = nn.Conv2d(out_size, out_size, kernel_size, padding=1)
        self.activation = activation

    def forward(self, x, bridge):
        up = self.up(x)
        out = torch.cat([up, bridge], 1)
        out = self.activation(self.conv(out))
        out = self.conv2(out)
        return out

In [13]:
class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()

        self.activation = F.tanh

        self.pool1 = nn.MaxPool2d(2)
        self.pool2 = nn.MaxPool2d(2)
        self.pool3 = nn.MaxPool2d(2)
        self.pool4 = nn.MaxPool2d(2)

        self.conv_block3_16 = UNetConvBlock(3, 16)
        self.conv_block16_32 = UNetConvBlock(16, 32)
        self.conv_block32_64 = UNetConvBlock(32, 64)
        self.conv_block64_128 = UNetConvBlock(64, 128)

        self.up_block128_64 = UNetUpBlock(128, 64)
        self.up_block64_32 = UNetUpBlock(64, 32)
        self.up_block32_16 = UNetUpBlock(32, 16)

        self.last = nn.Conv2d(16, 1, 1)

    def forward(self, x, want_print):

        if want_print:
            print("original shape : {}".format(x.shape))
        block1 = self.conv_block3_16(x)

        if want_print:
            print("block1 shape = {}".format(block1.shape))
        pool1 = self.pool1(block1)

        if want_print:
            print("pool1 shape = {}".format(pool1.shape))

        block2 = self.conv_block16_32(pool1)
        if want_print:
            print("block2 shape = {}".format(block2.shape))

        pool2 = self.pool2(block2)
        if want_print:
            print("pool2 shape = {}".format(pool2.shape))

        block3 = self.conv_block32_64(pool2)
        if want_print:
            print("block3 shape = {}".format(block3.shape))
        pool3 = self.pool3(block3)
        if want_print:
            print("pool3 shape = {}".format(pool3.shape))

        block4 = self.conv_block64_128(pool3)
        if want_print:
            print("block4 shape = {}".format(block4.shape))

        up1 = self.activation(self.up_block128_64(block4, block3))
        if want_print:
            print("up1 shape = {}".format(up1.shape))

        up2 = self.activation(self.up_block64_32(up1, block2))
        if want_print:
            print("up2 shape = {}".format(up2.shape))

        up3 = self.up_block32_16(up2, block1)
        if want_print:
            print("up3 shape = {}".format(up3.shape))
        # print(self.last(up3))
        return self.last(up3)

In [14]:
model = UNet()
model.to(device)

UNet(
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (pool4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv_block3_16): UNetConvBlock(
    (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (conv_block16_32): UNetConvBlock(
    (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (conv_block32_64): UNetConvBlock(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (conv_block64_128): UNetConvBlock(
    (conv): Conv2d(64, 1

In [15]:
def custom_loss_function(output, target):
    di = target - output
    n = (output_height * output_width)
    di2 = torch.pow(di, 2)
    fisrt_term = torch.sum(di2,(1,2,3))/n
    second_term = 0.5*torch.pow(torch.sum(di,(1,2,3)), 2)/ (n**2)
    loss = fisrt_term - second_term
    loss = fisrt_term
    return loss.mean()

In [16]:
# optimizer = optim.SGD(model.parameters(), lr = 0.1, momentum=0.9)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
save_frequency = 5


In [17]:
def train_Unet(epoch):
    model.train()
    train_coarse_loss = 0
    epoch_gradient = {}
    for batch_idx, image in enumerate(train_loader):
#        start = time.time()
        # pdb.set_trace()
        x = image['image'].to(device)
        y = image['depth'].to(device)

        optimizer.zero_grad()
        y_hat = model(x.to(device).to(torch.float32), want_print = False)
        # sys.exit()

        loss = custom_loss_function(y_hat, y)


        # loss = torch.nn.functional.mse_loss(y_hat, y)
        # print("loss = {}".format(loss))
        loss.backward()
        optimizer.step()
        train_coarse_loss += loss.item()


        for name, param in model.named_parameters():
            if param.grad == None:
                epoch_gradient[name + 'zero'] = 1
            elif name not in epoch_gradient:
                epoch_gradient[name] = param.grad.clone()
            else:
                epoch_gradient[name] += param.grad
        # print("train_coarse_loss = {}".format(train_coarse_loss))

    train_coarse_loss /= (batch_idx + 1)
    if epoch % save_frequency == 0 or epoch == epochs:
        checkpoint = {
                'epoch': epoch,
                'model_state_dict': model.state_dict(), # model.state_dict()是存下param的的值和形狀
                'optimizer_state_dict': optimizer.state_dict(), # optimizer.state_dict()則是存下優化器的param如momentum等等 不包含當下梯度
                'loss' : train_coarse_loss, # 記得不能存tensor
                'gradients' : epoch_gradient
            }
        torch.save(checkpoint, 'weight{}.pth'.format(epoch))
        source_path = 'weight{}.pth'.format(epoch)
        destination_path = '/content/drive/MyDrive/Colab Notebooks/Unet_Depth_Estimation/Checkpoint'
        shutil.copy(source_path, destination_path)
    return train_coarse_loss


# print("Epochs:     Train_loss  Val_loss    Delta_1     Delta_2     Delta_3    rmse_lin    rmse_log    abs_rel.  square_relative")
# print("Paper Val:                          (0.618)     (0.891)     (0.969)     (0.871)     (0.283)     (0.228)     (0.223)")

In [28]:
def warmup_scheduler(epoch):
    if epoch < 10:
        return (epoch / 10)
    else:
        return 1 / 20

In [29]:
scheduler = LambdaLR(optimizer, lr_lambda = warmup_scheduler)

In [23]:
def train_Unet_cont(epoch):

    model.train()
    train_coarse_loss = 0
    epoch_gradient = {}
    for batch_idx, image in enumerate(train_loader):
        scheduler.step(epoch)
#        start = time.time()
        # pdb.set_trace()
        x = image['image'].to(device)
        y = image['depth'].to(device)

        optimizer.zero_grad()
        y_hat = model(x.to(device).to(torch.float32), want_print = False)
        # sys.exit()

        loss = custom_loss_function(y_hat, y)


        # loss = torch.nn.functional.mse_loss(y_hat, y)
        # print("loss = {}".format(loss))
        loss.backward()
        optimizer.step()
        train_coarse_loss += loss.item()


        for name, param in model.named_parameters():
            if param.grad == None:
                epoch_gradient[name + 'zero'] = 1
            elif name not in epoch_gradient:
                epoch_gradient[name] = param.grad.clone()
            else:
                epoch_gradient[name] += param.grad
        # print("train_coarse_loss = {}".format(train_coarse_loss))

    train_coarse_loss /= (batch_idx + 1)
    if epoch % save_frequency == 0 or epoch == epochs:
        checkpoint = {
                'epoch': epoch,
                'model_state_dict': model.state_dict(), # model.state_dict()是存下param的的值和形狀
                'optimizer_state_dict': optimizer.state_dict(), # optimizer.state_dict()則是存下優化器的param如momentum等等 不包含當下梯度
                'loss' : train_coarse_loss, # 記得不能存tensor
                'gradients' : epoch_gradient
            }
        torch.save(checkpoint, 'weight{}.pth'.format(epoch))
        source_path = 'weight{}.pth'.format(epoch)
        destination_path = '/content/drive/MyDrive/Colab Notebooks/Unet_Depth_Estimation/Checkpoint'
        shutil.copy(source_path, destination_path)
    return train_coarse_loss


# print("Epochs:     Train_loss  Val_loss    Delta_1     Delta_2     Delta_3    rmse_lin    rmse_log    abs_rel.  square_relative")
# print("Paper Val:                          (0.618)     (0.891)     (0.969)     (0.871)     (0.283)     (0.228)     (0.223)")

In [18]:
# epochs = 200
# for epoch in range(1, epochs + 1):
#     training_loss = train_Unet(epoch)
#     print("epoch : {} | training loss : {}".format(epoch, training_loss))

    # if epoch % 1 == 0:
    #     validate_Unet(epoch, training_loss)
    # if epoch % 1 == 0:
    #     model_file = folder_name + "/model_" + str(epoch) + ".pth"
    #     torch.save(model.state_dict(), model_file)

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


epoch : 1 | training loss : 2.6875822730362415
epoch : 2 | training loss : 1.9591245017945766
epoch : 3 | training loss : 1.9641172252595425
epoch : 4 | training loss : 1.8840868920087814
epoch : 5 | training loss : 1.8811348639428616
epoch : 6 | training loss : 2.0353343077003956
epoch : 7 | training loss : 1.9110196754336357
epoch : 8 | training loss : 1.9047838561236858
epoch : 9 | training loss : 1.8526328019797802
epoch : 10 | training loss : 1.8220750242471695
epoch : 11 | training loss : 1.8431522585451603
epoch : 12 | training loss : 1.959611602127552
epoch : 13 | training loss : 1.8121162503957748
epoch : 14 | training loss : 1.822212852537632
epoch : 15 | training loss : 1.8442072086036205
epoch : 16 | training loss : 1.871311392635107
epoch : 17 | training loss : 1.8316975384950638
epoch : 18 | training loss : 1.7602797374129295
epoch : 19 | training loss : 1.7434262409806252
epoch : 20 | training loss : 1.8051424957811832
epoch : 21 | training loss : 1.842110887169838
epoch

KeyboardInterrupt: 

In [30]:
epochs = 600
start = 485
weight_path = '/content/drive/MyDrive/Colab Notebooks/Unet_Depth_Estimation/Checkpoint/weight{}.pth'.format(start)
checkpoint = torch.load(weight_path, map_location = torch.device(device))
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
for epoch in range(start + 1, epochs + 1):
    training_loss = train_Unet_cont(epoch)
    print("epoch : {} | training loss : {}".format(epoch, training_loss))

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:143: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this 

epoch : 486 | training loss : 1.1593000963330269
epoch : 487 | training loss : 1.153380323201418
epoch : 488 | training loss : 1.1528622601181269
epoch : 489 | training loss : 1.1524217836558819
epoch : 490 | training loss : 1.1517167147248983
epoch : 491 | training loss : 1.1520654447376728
epoch : 492 | training loss : 1.1503383070230484
epoch : 493 | training loss : 1.150115618482232
epoch : 494 | training loss : 1.1502727549523115
epoch : 495 | training loss : 1.1525887101888657
epoch : 496 | training loss : 1.149732980877161
epoch : 497 | training loss : 1.1496012173593044
epoch : 498 | training loss : 1.1507580317556858
epoch : 499 | training loss : 1.1501389499753714
epoch : 500 | training loss : 1.1501252856105566
epoch : 501 | training loss : 1.1492851059883833
epoch : 502 | training loss : 1.1497801821678877
epoch : 503 | training loss : 1.1492963302880526
epoch : 504 | training loss : 1.149484595283866
epoch : 505 | training loss : 1.1504611801356077
epoch : 506 | training l

KeyboardInterrupt: 

In [31]:
from google.colab import runtime
runtime.unassign()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')